In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import gc
import time

import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
path = '../input/'
# Any results you write to the current directory are saved as output.

In [4]:
# 读取训练集和测试集
train = pd.read_csv(path + 'atec_anti_fraud_train.csv')
train = train[train.label.isin([0, 1])]
test = pd.read_csv(path + 'atec_anti_fraud_test_a.csv')

In [5]:
# 合并train与test，统一处理
test['label'] = -1
data = pd.concat((train, test), axis=0, sort=True)

In [6]:
del train, test
gc.collect()

In [7]:
data.head()

In [8]:
# 处理date，年、月、日
data['year'] = data['date'].apply(lambda x: int(str(x)[0:4]))
data['month'] = data['date'].apply(lambda x: int(str(x)[4:6]))
data['day'] = data['date'].apply(lambda x: int(str(x)[6:8]))

In [9]:
# 得到int，float的列名
float_cols = data.blocks['float64'].columns
int_cols = data.blocks['int64'].columns

In [10]:
d1 = dict((col, -1) for col in int_cols)
d2 = dict((col, -1.) for col in float_cols)

In [11]:
# 处理缺失值
data = data.fillna(value=dict(d1, **d2))

In [12]:
train = data[data.label.isin([0, 1])]
test = data[data.label.isin([-1])]

del data
gc.collect()

In [13]:
train.groupby('month').count()['date']

In [14]:
gc.collect()

In [15]:
train = train.drop('id', axis=1)
res = test[['id']]
test = test.drop(['id', 'label'], axis=1)

val = train[train['month'] == 11]
train = train[train['month'] != 11]
train_label = train.pop('label')
val_label = val.pop('label')

In [16]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return score

In [30]:
def rf_model(train_x, train_y, val_x, val_y, res, test):
#     for i in range(13, 16, 2):
    for i in range(1):
        start_time = time.time()
        clf = RandomForestClassifier(
            n_estimators=16, 
            max_depth=15, 
            criterion='entropy', 
            max_features=20,
            min_samples_split=402,
            min_samples_leaf=1,
#             max_leaf_nodes=i,
            random_state=0, 
            n_jobs=4, 
            class_weight={0: 1, 1: 1})
        clf.fit(train_x, train_y)
        train_proba = clf.predict_proba(train_x)[:, 1]
        val_proba = clf.predict_proba(val_x)[:, 1]
        train_score = eval_func(train_y, train_proba)
        val_score = eval_func(val_y, val_proba)
        
        cost_time = time.time() - start_time
        
        print('max_leaf_nodes = ', i, ' train score : ', train_score, ' validation score : ', val_score, ' diff = ', train_score - val_score, ' cost time : ', cost_time)
        
    res['score'] = clf.predict_proba(test)[:,1]
    # res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('submission.csv', index=False)
    return clf

In [ ]:
clf = rf_model(train, train_label, val, val_label, res, test)